Import essential libraries

In [358]:
import pandas as pd

Load all streaming platform and rating csvs (Amazon Prime, Netflix, Disney Plus, Hulu and all ratings files)

In [359]:
amzdf = pd.read_csv('./datasets/amazon_prime_titles.csv')
netdf= pd.read_csv('./datasets/netflix_titles.csv')
disdf = pd.read_csv('./datasets/disney_plus_titles.csv')
hudf = pd.read_csv('./datasets/hulu_titles.csv')
r1 = pd.read_csv('./datasets/ratings/1.csv', index_col='movieId')
r2 = pd.read_csv('./datasets/ratings/2.csv', index_col='movieId')
r3 = pd.read_csv('./datasets/ratings/3.csv', index_col='movieId')
r4 = pd.read_csv('./datasets/ratings/4.csv', index_col='movieId')
r5 = pd.read_csv('./datasets/ratings/5.csv', index_col='movieId')
r6 = pd.read_csv('./datasets/ratings/6.csv', index_col='movieId')
r7 = pd.read_csv('./datasets/ratings/7.csv', index_col='movieId')
r8 = pd.read_csv('./datasets/ratings/8.csv', index_col='movieId')

Modify all index to begin with the letter of the platform

In [360]:
amzdf['show_id'] = amzdf['show_id'].apply(lambda x: 'a' + x)
amzdf.set_index('show_id', inplace=True)
netdf['show_id'] = netdf['show_id'].apply(lambda x: 'n' + x)
netdf.set_index('show_id', inplace=True)
disdf['show_id'] = disdf['show_id'].apply(lambda x: 'd' + x)
disdf.set_index('show_id', inplace=True)
hudf['show_id'] = hudf['show_id'].apply(lambda x: 'h' + x)
hudf.set_index('show_id', inplace=True)

Then we merge all csvs for all streming products and all the ratings into one

In [361]:
streaming = pd.concat([amzdf, netdf, disdf, hudf])
ratings = pd.concat([r1, r2, r3, r4, r5, r6, r7, r8])

We replace null values on rating column to 'g' rating

In [362]:
streaming['rating'].fillna('G', inplace=True)


Change date format on 'date_added' column from 'Month Day, Year' to 'YYYY-DD-MM'

In [363]:
streaming['date_added'] = pd.to_datetime(streaming.date_added)

Split into two columns ('duration_int' and 'duration_type') the 'duration' column

In [365]:
streaming[['duration_int', 'duration_type']] = streaming['duration'].str.split(' ', 1, expand=True)
streaming.drop('duration', axis =1 , inplace=True)

/var/folders/69/7m4xz9b13hj96zbkt5lxyytr0000gn/T/ipykernel_16526/1496341358.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  streaming[['duration_int', 'duration_type']] = streaming['duration'].str.split(' ', 1, expand=True)


Fill with 1 all null values and set column type to int

In [366]:
streaming['duration_int'].fillna(1, inplace=True)
streaming['duration_int'].astype(int)

show_id
as1       113
as2       110
as3        74
as4        69
as5        45
         ... 
hs3069      3
hs3070      7
hs3071      1
hs3072      5
hs3073      1
Name: duration_int, Length: 22998, dtype: int64

Make the whole dataset into lowercase

In [367]:
streaming = streaming.applymap(lambda s: s.lower() if type(s) == str else s)
streaming.head()

,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type
show_id,,,,,,,,,,,,
as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min
as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110,min
as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,74,min
as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",69,min
as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,45,min


Set the index as the column 'show_id'

In [369]:
ratings.rename_axis('show_id')
#ratings.set_index('show_id', inplace=True)

,userId,rating,timestamp
show_id,,,
as680,1,1.0,1425941529
ns2186,1,4.5,1425942435
hs2381,1,5.0,1425941523
ns3663,1,5.0,1425941546
as9500,1,5.0,1425941556
...,...,...,...
ns5272,124380,4.5,1196786159
ns5492,124380,2.5,1196786030
hs305,124380,3.5,1196785679


On the ratings dataframe we delete the column 'userId' and 'timestamp'

In [371]:
ratings = ratings.drop(columns=['userId', 'timestamp'])

Group all the ratings by movieId

In [373]:
rating_group = ratings.groupby('movieId').mean()

Rename the index of ratings dataset to 'show_id' and then round to two decimals all rating values

In [374]:
rating_group = rating_group.rename_axis('show_id')
rating_group['rating'] = rating_group['rating'].round(decimals=2)
rating_group.head()

,rating
show_id,
as1,3.47
as10,3.44
as100,3.61
as1000,3.56
as1001,3.59


Merge both dataframes to make a final usable dataset

In [375]:
result = pd.merge(streaming, rating_group,left_index=True, right_index=True, how='left')
result.head()

,type,title,director,cast,country,date_added,release_year,rating_x,listed_in,description,duration_int,duration_type,rating_y
show_id,,,,,,,,,,,,,
as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min,3.47
as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110,min,3.55
as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,74,min,3.50
as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",69,min,3.54
as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,45,min,3.48


Since when merging both dataframes we have two columns with same names, we rename them as rating and score

In [376]:
result = result.rename(columns={'rating_y': 'score'})
result = result.rename(columns={'rating_x': 'rating'})
result.head()

,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type,score
show_id,,,,,,,,,,,,,
as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min,3.47
as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110,min,3.55
as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,74,min,3.50
as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",69,min,3.54
as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,45,min,3.48


Finally, export the dataset

In [377]:
result.to_csv('./datasets/streaming.csv')